In [1]:
import numpy as np
from keras.layers import LSTM,RepeatVector,Dense, Embedding,Activation, Dot, Concatenate,Bidirectional,Input
from keras.callbacks import ModelCheckpoint
from keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import AdditiveAttention
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


Using TensorFlow backend.


In [2]:
#importing vocabulary
ds_path='/home/al/Desktop/nnfl/word2vecmod.txt'
data = np.loadtxt(ds_path,dtype=str,usecols=(0))
data=np.append(data,[' '])
print(data)
print(data.shape[0])

['a' 'abandon' 'ability' ... 'yield' 'youngster' ' ']
5024


In [77]:
tmp_file = get_tmpfile(datapath("/home/al/Desktop/nnfl/wvc"))
word_model = KeyedVectors.load(tmp_file)


In [69]:
#making word-to-index and index-to-word dictionaries
word_to_ix=dict(zip(data,range(0,data.shape[0])))
ix_to_word=dict(zip(range(0,data.shape[0]),data))

In [83]:
#constants
voc=data.shape[0]+3#for <start>,<end>,<pad>
ip_len=20
op_len=20
n_s=64+op_len#no of decoder lstm
n_a=32#no 

1424


In [91]:
#one-hot function
def oh(Xq,ip_ln):
    Xoh=np.zeros([Xq.shape[0],ip_ln,voc])
    for i in range(Xq.shape[0]):
        for j in range(ip_ln):
            Xoh[i,j,int(Xq[i,j])]=1
    return Xoh

In [92]:
#creating checkpoint
filepath="/home/al/Desktop/nnfl final/weightsfin.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_accuracy',verbose=1, save_best_only=True, mode='max')

In [98]:
d_path='/home/al/Desktop/nnflproj/nds/nds-a'
idx=0
for e  in 'a':
#bcdefghi':
    for f  in 'a':
        #bcdefghijklmnopqrstuvwxyz':
        ds = np.loadtxt(d_path+e+f+'.csv',delimiter='\n',dtype='str')
        num_sen=ds.shape[0]
        idx=0
        Xtr=np.ndarray(shape=[num_sen,ip_len])
        for j in range(0,int(num_sen)):
                #ds.shape[0]-(ds.shape[0]%1000)+1)
                w1=np.array(ds[j].lower().split())
                for i in range(0,ip_len):    
                    w=w1[i]
                    if i==ip_len-1:
                        #w1_sum/=50
                        Xtr[j,i]=voc-2#for<end>
                        
                        break
                    if w=='<start>':
                        Xtr[j,i]=voc-3
                        continue
                    if w=='<end>':
                        while i<ip_len-1:
                            Xtr[j,i]=voc-1#padding
                            i+=1
                        Xtr[j,i]=voc-2#for<end>
                        break
                    if w in data:                
                        Xtr[j,i]=word_to_ix[w]
                   
                    elif w in data2:
                            temp=word_model.most_similar(positive=[w],negative=[],restrict_vocab=4095)#for<unk>
                        
                            Xtr[j,i]=word_to_ix[temp[0][0]]

                    else:
                           if(len(w)>1):
                                Xtr[j,i]=word_to_ix['it']
                           else:
                                Xtr[j,i]=word_to_ix[' ']
                           
                           idx+=1
                           
                if j%1000==0:
                    print(j)
    print(idx)


0
0
shindang
1
bupyung
2
aaggh
3
bupyung
4
bupyung
5
bupyung
6
jamonka


KeyboardInterrupt: 

In [95]:
enc_ip=Input(shape=(ip_len,voc))
enc_lstm=LSTM(32,return_state=True)
bi=Bidirectional(enc_lstm)
enc_op,h1,c1,h2,c2=bi(enc_ip)
concat=Concatenate(axis=-1)
h=concat([h1,h2])
c=concat([c1,c2])
enc_states=[h,c]
att=AdditiveAttention([c,enc_op])
dec_ip=Input(shape=(op_len,voc))
dec_lstm=LSTM(64,return_sequences=True,return_state=True)
dec_op,_,_=dec_lstm(dec_ip,initial_state=enc_states)
dense=Dense(voc,activation='softmax')
dec_op=dense(dec_op)
model=Model([enc_ip,dec_ip],dec_op)
model.compile(optimizer="rmsprop",loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 20, 5027)     0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) [(None, 64), (None,  1295360     input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 20, 5027)     0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 64)           0           bidirectional_2[0][1]            
                                                                 bidirectional_2[0][3]      

In [96]:
for kl in range(64,10000,64):
    u=oh(Xtr[kl-64:kl+1],ip_len)
    model.fit([u[:-1],u[:-1]],u[1:],epochs=100,steps_per_epoch=4,validation_split=0.2,validation_steps=4,callbacks=[checkpoint])
    model.load_weights(filepath)

Train on 51 samples, validate on 13 samples
Epoch 1/100
4/4 [==============================] - 5s 1s/step - loss: 8.4917 - accuracy: 0.6191 - val_loss: 2.0948 - val_accuracy: 4.0000

Epoch 00001: val_accuracy improved from -inf to 4.00000, saving model to /home/al/Desktop/nnfl/weightsv2.hdf5
Epoch 2/100
4/4 [==============================] - 2s 464ms/step - loss: 8.0494 - accuracy: 0.6738 - val_loss: 1.7157 - val_accuracy: 4.0000

Epoch 00002: val_accuracy did not improve from 4.00000
Epoch 3/100
4/4 [==============================] - 2s 446ms/step - loss: 6.5509 - accuracy: 0.6500 - val_loss: 1.3854 - val_accuracy: 4.0000

Epoch 00003: val_accuracy did not improve from 4.00000
Epoch 4/100
4/4 [==============================] - 2s 444ms/step - loss: 5.4606 - accuracy: 0.6500 - val_loss: 1.1515 - val_accuracy: 4.0000

Epoch 00004: val_accuracy did not improve from 4.00000
Epoch 5/100
4/4 [==============================] - 2s 444ms/step - loss: 4.6574 - accuracy: 0.6500 - val_loss: 0.959

KeyboardInterrupt: 

In [ ]:
#change vocab size and do in another weights file

In [18]:
ut=oh(Xtr[0:1],ip_len)
a=model.predict([ut,ut])
b=np.argmax(a,axis=-1)
print(b)
for el in b.flatten():
    if el>999:
        print('<unk>')
    else:
        print(ix_to_word[el])
print(Xtr[0])

[[3019 3019 3019 3019 3019 3017 3017 3017 3017 3017 3017 3017 3017 3017
  3017 3017 3017 3017 3017 3017]]
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
[3019  939 2805 3019   68 2735 3019 2389  105 3019 3019    0 2728 3019
 1252 3019 3017 3017 3017 3018]


In [75]:
word_model.save('/home/al/Desktop/nnfl/wvc')

In [ ]:
data4=np.loadtxt('/home/al/Desktop/nnfl/wvec.txt',dtype=str,usecols=(0),skiprows=1)

In [60]:
data4=np.loadtxt('/home/al/Desktop/nnfl/wvec.txt',dtype=float,usecols=(range(1,101)),skiprows=1)

In [74]:
word_model.add(entities=data2[-10:],weights=data4[-10:],replace=True)

In [61]:
word_to_ix2=dict(zip(data2,range(0,data2.shape[0])))
ix_to_word2=dict(zip(range(0,data2.shape[0]),data2))
a1=data4[word_to_ix2['is']]-data4[word_to_ix2['do']]+data4[word_to_ix2['did']]

In [58]:
is11=data4[word_to_ix2['say']]+data4[word_to_ix2['did']]-data4[word_to_ix2['do']]

In [59]:
print(is11 )

[ 3.832780e-01 -1.893000e-02  3.963300e-01 -1.417100e-01 -4.385500e-01
 -6.724300e-01  3.265160e-01 -3.494100e-01  2.415100e-01  3.086500e-02
 -6.436000e-02  6.528940e-01  3.027890e-01 -4.417100e-01 -2.190500e-01
 -8.316000e-02 -6.100500e-02 -1.612800e-01 -8.535200e-01  2.150300e-01
  5.907600e-01 -1.142460e-01  1.232230e-01  6.340000e-04 -3.086400e-01
 -2.389100e-01 -4.024800e-01 -4.911800e-01  3.280000e-02 -4.476400e-01
  9.883700e-01  1.016700e+00 -2.558710e-01  5.569680e-01 -3.100850e-01
  3.403000e-02 -9.082660e-02  8.380700e-01  1.837000e-01  1.196230e-01
 -1.275400e+00 -1.351900e-01  4.608650e-01  2.362000e-02 -1.060800e-01
 -7.905890e-01  5.569000e-01 -4.972000e-01  6.038100e-01 -1.033900e+00
  5.537390e-01  7.305400e-02  2.948200e-01  1.029880e+00 -5.145858e-01
 -2.171600e+00 -5.945430e-01 -4.352970e-01  9.988000e-01  1.274070e+00
 -4.341900e-01  4.573700e-01  2.804560e-01  1.570860e-01  5.620200e-01
 -5.832470e-01  3.816300e-01  8.388600e-01 -5.577800e-01  3.734400e-01
 -1.12